<a href="https://colab.research.google.com/github/nicolaiberk/nlpdl_project/blob/main/01_Training_PPRs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install torch
!pip install sklearn
!pip install wandb

# code is based on Hauke Licht's CAP Model (https://colab.research.google.com/drive/1n7yHr0-lq-hmsXe2sqxLUhq7_4ejLH9o?usp=sharing#scrollTo=CiaoE0V8XZc6)

     |████████████████████████████████| 2.1MB 19.6MB/s 
     |████████████████████████████████| 901kB 54.1MB/s 
     |████████████████████████████████| 3.3MB 51.4MB/s 
     |████████████████████████████████| 2.1MB 18.2MB/s 
     |████████████████████████████████| 133kB 52.4MB/s 
     |████████████████████████████████| 163kB 46.5MB/s 
     |████████████████████████████████| 102kB 14.5MB/s 
     |████████████████████████████████| 71kB 10.9MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8786 sha256=fdf3aebf510b0c27a77ebb5aa1a59f61fe21cec0f2fc1cf6139b941139e17e4b
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6489 sha256=e50d52f4e2cbb8198c0b3b44eda369a77e9f2a7c4ab9e5f6b88a97ea4b4e1740
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
Successfully built pa

In [ ]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
%env WANDB_PROJECT=press_releases_classifier

env: WANDB_PROJECT=press_releases_classifier


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# load and prepare data
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/nlp_project_nico/data/press_releases_lean.csv")
df = df.reset_index()
df = df.dropna()
df = df.sample(frac=0.10, random_state=123) # drop this for full model


In [ ]:
# Creating Train, Valid and Test dataset

train_size = 0.9
train = df.sample(frac=train_size,random_state=123)
test = df.drop(train.index).reset_index(drop=True)
train = train.reset_index(drop=True)

print("FULL Dataset: {}".format(df.shape))
print("TRAIN Dataset: {}".format(train.shape))
print("TEST Dataset: {}".format(test.shape))

FULL Dataset: (2770, 3)
TRAIN Dataset: (2493, 3)
TEST Dataset: (277, 3)


In [ ]:
train_texts = list(train["text"])
train_labels = list(train["label"])

test_texts = list(test["text"])
test_labels = list(test["label"])


# there are probably better ways to do this
ulabels = list(set(train_labels))
train_label_dict = {}

for i in range(len(ulabels)):
  train_label_dict[str(ulabels[i])] = i

train_labels = [train_label_dict[str(l)] for l in train_labels]


# and for test set
ulabels = list(set(test_labels))
test_label_dict = {}

for i in range(len(ulabels)):
  test_label_dict[str(ulabels[i])] = i

test_labels = [test_label_dict[str(l)] for l in test_labels]

In [ ]:
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.2, random_state=1234, stratify=train_labels)

print(f"No. obs    --  train: {len(train_labels)}; validation: {len(val_labels)}; test: {len(test_labels)};")
print(f"No. labels --  train: {len(set(train_labels))}; validation: {len(set(val_labels))}; test: {len(set(test_labels))};")

No. obs    --  train: 1994; validation: 499; test: 277;
No. labels --  train: 6; validation: 6; test: 6;


In [ ]:
# load the tokenizer 
from transformers import DistilBertTokenizerFast

tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-german-cased")

# tokenize texts
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

In [ ]:
import torch
class PPRDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = PPRDataset(train_encodings, train_labels)
val_dataset = PPRDataset(val_encodings, val_labels)
test_dataset = PPRDataset(test_encodings, test_labels)

In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, n = precision_recall_fscore_support(labels, preds, average="macro")
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall,
        'n': n
    }



        Batch size: 16, 32
        Learning rate (Adam): 5e-5, 3e-5, 2e-5
        Number of epochs: 2, 3, 4



In [ ]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments, AdamW
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import os

In [ ]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments, AdamW
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import os

#"distilbert-base-high-lr"

training_args = TrainingArguments(
    # output directory
    output_dir=os.path.join("trained", "distilbert_cased", "results"),
    # total number of training epochs
    num_train_epochs=2,
    # batch size per device during training
    per_device_train_batch_size=16,
    # batch size for evaluation
    per_device_eval_batch_size=16,
    # number of warmup steps for learning rate scheduler
    warmup_steps=100,
    # strength of weight decay
    weight_decay=0.01, 
    # directory for storing logs
    logging_dir=os.path.join("trained", "distilbert_cased", "logs"),            
    logging_steps=50,
    evaluation_strategy = "steps",
    report_to="wandb",  # enable logging to W&B
    run_name="distilbert-base-high-lr"
)

model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-german-cased", num_labels = len(set(test_labels)))

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,

)

trainer.train()
wandb.finish()

Some weights of the model checkpoint at distilbert-base-german-cased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-german-cased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias

Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,N,Runtime,Samples Per Second
50,1.757700,1.676349,0.412826,0.326793,0.477813,0.388438,None,14.919300,33.447000
100,1.202700,0.617914,0.841683,0.835448,0.856529,0.835130,None,14.997200,33.273000
150,0.342400,0.141441,0.967936,0.966211,0.967037,0.966785,None,14.983200,33.304000
200,0.127800,0.075076,0.981964,0.981537,0.982130,0.981294,None,14.997700,33.272000
250,0.087800,0.069253,0.981964,0.981436,0.982068,0.981429,None,14.954300,33.368000


train/loss,0.0878
train/learning_rate,0.0
train/epoch,2.0
train/global_step,250
_runtime,431
_timestamp,1619971581
_step,10
eval/loss,0.06925
eval/accuracy,0.98196
eval/f1,0.98144
eval/precision,0.98207


train/loss,█▆▂▁▁
train/learning_rate,▅█▆▃▁
train/epoch,▁▁▃▃▅▅▆▆███
train/global_step,▁▁▃▃▅▅▆▆███
_runtime,▁▁▃▃▄▅▆▆███
_timestamp,▁▁▃▃▄▅▆▆███
_step,▁▂▂▃▄▅▅▆▇▇█
eval/loss,█▃▁▁▁
eval/accuracy,▁▆███
eval/f1,▁▆███
eval/precision,▁▆███


In [ ]:
#"distilbert-base-mid-lr"

training_args = TrainingArguments(
    # output directory
    output_dir=os.path.join("trained", "distilbert_cased", "results"),
    # total number of training epochs
    num_train_epochs=2,
    # batch size per device during training
    per_device_train_batch_size=16,
    # batch size for evaluation
    per_device_eval_batch_size=32,
    # number of warmup steps for learning rate scheduler
    warmup_steps=100,
    # strength of weight decay
    weight_decay=0.01, 
    # learning rate
    learning_rate=3e-5,
    # directory for storing logs
    logging_dir=os.path.join("trained", "distilbert_cased", "logs"),            
    logging_steps=50,
    evaluation_strategy = "steps",
    report_to="wandb",  # enable logging to W&B
    run_name="distilbert-base-mid-lr"
)

model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-german-cased", num_labels = len(set(test_labels)))

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

trainer.train()
wandb.finish()

Some weights of the model checkpoint at distilbert-base-german-cased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-german-cased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias

Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,N,Runtime,Samples Per Second
50,1.775800,1.695046,0.254509,0.119344,0.442271,0.202406,None,15.616500,31.953000
100,1.417900,0.893651,0.785571,0.761526,0.826057,0.775257,None,15.623800,31.938000
150,0.751500,0.460860,0.879760,0.874572,0.884412,0.876083,None,15.594900,31.998000
200,0.460800,0.317088,0.909820,0.906978,0.913080,0.907764,None,15.620800,31.945000
250,0.320600,0.272045,0.917836,0.915001,0.918459,0.916152,None,15.684000,31.816000


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


train/loss,0.3206
train/learning_rate,0.0
train/epoch,2.0
train/global_step,250
_runtime,451
_timestamp,1619776655
_step,10
eval/loss,0.27205
eval/accuracy,0.91784
eval/f1,0.915
eval/precision,0.91846


train/loss,█▆▃▂▁
train/learning_rate,▅█▆▃▁
train/epoch,▁▁▃▃▅▅▆▆███
train/global_step,▁▁▃▃▅▅▆▆███
_runtime,▁▁▃▃▄▅▆▆███
_timestamp,▁▁▃▃▄▅▆▆███
_step,▁▂▂▃▄▅▅▆▇▇█
eval/loss,█▄▂▁▁
eval/accuracy,▁▇███
eval/f1,▁▇███
eval/precision,▁▇▇██


In [ ]:
#"distilbert-base-low-lr"

training_args = TrainingArguments(
    # output directory
    output_dir=os.path.join("trained", "distilbert_cased", "results"),
    # total number of training epochs
    num_train_epochs=2,
    # batch size per device during training
    per_device_train_batch_size=16,
    # batch size for evaluation
    per_device_eval_batch_size=32,
    # number of warmup steps for learning rate scheduler
    warmup_steps=100,
    # strength of weight decay
    weight_decay=0.01, 
    # learning rate
    learning_rate=2e-5,
    # directory for storing logs
    logging_dir=os.path.join("trained", "distilbert_cased", "logs"), 
    evaluation_strategy = "steps",           
    logging_steps=50,
    report_to="wandb",  # enable logging to W&B
    run_name="distilbert-base-low-lr"
)

model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-german-cased", num_labels = len(set(test_labels)))

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

trainer.train()
wandb.finish()

Some weights of the model checkpoint at distilbert-base-german-cased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-german-cased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias

Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,N,Runtime,Samples Per Second
50,1.784400,1.729977,0.240481,0.099582,0.265237,0.188074,None,15.645900,31.893000
100,1.570000,1.217173,0.717435,0.677312,0.763511,0.703979,None,15.675100,31.834000
150,0.991700,0.669919,0.851703,0.845228,0.859706,0.846045,None,15.576600,32.035000
200,0.657600,0.499262,0.869739,0.865039,0.877428,0.865843,None,15.661600,31.861000
250,0.529400,0.445700,0.887776,0.884148,0.894253,0.884834,None,15.620500,31.945000


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


train/loss,0.5294
train/learning_rate,0.0
train/epoch,2.0
train/global_step,250
_runtime,451
_timestamp,1619777115
_step,10
eval/loss,0.4457
eval/accuracy,0.88778
eval/f1,0.88415
eval/precision,0.89425


train/loss,█▇▄▂▁
train/learning_rate,▅█▆▃▁
train/epoch,▁▁▃▃▅▅▆▆███
train/global_step,▁▁▃▃▅▅▆▆███
_runtime,▁▁▃▃▄▅▆▆███
_timestamp,▁▁▃▃▄▅▆▆███
_step,▁▂▂▃▄▅▅▆▇▇█
eval/loss,█▅▂▁▁
eval/accuracy,▁▆███
eval/f1,▁▆███
eval/precision,▁▇███


In [ ]:
#"distilbert-3-epochs-high-lr"

training_args = TrainingArguments(
    # output directory
    output_dir=os.path.join("trained", "distilbert_cased", "results"),
    # total number of training epochs
    num_train_epochs=3,
    # batch size per device during training
    per_device_train_batch_size=16,
    # batch size for evaluation
    per_device_eval_batch_size=32,
    # number of warmup steps for learning rate scheduler
    warmup_steps=100,
    # strength of weight decay
    weight_decay=0.01, 
    # learning rate
    learning_rate=5e-5,
    # directory for storing logs
    logging_dir=os.path.join("trained", "distilbert_cased", "logs"),            
    logging_steps=50,
    evaluation_strategy = "steps",
    report_to="wandb",  # enable logging to W&B
    run_name="distilbert-3-epochs-high-lr"
)

model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-german-cased", num_labels = len(set(test_labels)))

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

trainer.train()
wandb.finish()

Some weights of the model checkpoint at distilbert-base-german-cased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-german-cased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias

Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,N,Runtime,Samples Per Second
50,1.756100,1.588751,0.432866,0.348077,0.582424,0.390766,None,15.664200,31.856000
100,1.151100,0.583030,0.839679,0.827924,0.853402,0.833140,None,15.648200,31.889000
150,0.474000,0.225297,0.945892,0.943113,0.943695,0.943102,None,15.733700,31.715000
200,0.167000,0.091802,0.973948,0.972496,0.972639,0.973238,None,15.640500,31.904000
250,0.070900,0.054093,0.981964,0.981242,0.981713,0.981247,None,15.597000,31.993000
300,0.029800,0.047798,0.989980,0.989671,0.989447,0.989992,None,15.641700,31.902000
350,0.042400,0.041256,0.989980,0.989671,0.989447,0.989992,None,15.594300,31.999000


train/loss,0.0424
train/learning_rate,0.0
train/epoch,3.0
train/global_step,375
_runtime,667
_timestamp,1619777792
_step,14
eval/loss,0.04126
eval/accuracy,0.98998
eval/f1,0.98967
eval/precision,0.98945


train/loss,█▆▃▂▁▁▁
train/learning_rate,▄█▇▅▄▂▁
train/epoch,▁▁▂▂▃▃▄▄▅▅▆▆▇▇█
train/global_step,▁▁▂▂▃▃▄▄▅▅▆▆▇▇█
_runtime,▁▁▂▂▃▃▄▄▅▅▆▇▇██
_timestamp,▁▁▂▂▃▃▄▄▅▅▆▇▇██
_step,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
eval/loss,█▃▂▁▁▁▁
eval/accuracy,▁▆▇████
eval/f1,▁▆▇████
eval/precision,▁▆▇████


In [ ]:
#"distilbert-4-epochs-high-lr"

training_args = TrainingArguments(
    # output directory
    output_dir=os.path.join("trained", "distilbert_cased", "results"),
    # total number of training epochs
    num_train_epochs=4,
    # batch size per device during training
    per_device_train_batch_size=16,
    # batch size for evaluation
    per_device_eval_batch_size=32,
    # number of warmup steps for learning rate scheduler
    warmup_steps=100,
    # strength of weight decay
    weight_decay=0.01, 
    # learning rate
    learning_rate=5e-5,
    # directory for storing logs
    logging_dir=os.path.join("trained", "distilbert_cased", "logs"),            
    logging_steps=50,
    evaluation_strategy = "steps",
    report_to="wandb",  # enable logging to W&B
    run_name="distilbert-4-epochs-high-lr"
)

model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-german-cased", num_labels = len(set(test_labels)))

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

trainer.train()
wandb.finish()

Some weights of the model checkpoint at distilbert-base-german-cased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-german-cased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias

Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,N,Runtime,Samples Per Second
50,1.755800,1.599021,0.374749,0.237479,0.326800,0.328894,None,15.710000,31.763000
100,1.152900,0.582796,0.851703,0.846202,0.859802,0.844943,None,15.690600,31.802000
150,0.397000,0.111413,0.977956,0.977032,0.977249,0.976920,None,15.686500,31.811000
200,0.069900,0.046069,0.989980,0.989337,0.989077,0.989841,None,15.605600,31.976000
250,0.040900,0.032697,0.993988,0.993791,0.993567,0.994060,None,15.681600,31.821000
300,0.011500,0.037536,0.989980,0.989353,0.989116,0.989636,None,15.592200,32.003000
350,0.018700,0.036347,0.989980,0.989328,0.989041,0.989636,None,15.696200,31.791000
400,0.014000,0.039869,0.989980,0.989353,0.989116,0.989636,None,15.651000,31.883000
450,0.005800,0.047061,0.985972,0.984996,0.984783,0.985416,None,15.609000,31.969000
500,0.002400,0.043183,0.985972,0.984996,0.984783,0.985416,None,15.650200,31.885000


train/loss,0.0024
train/learning_rate,0.0
train/epoch,4.0
train/global_step,500
_runtime,899
_timestamp,1619778700
_step,20
eval/loss,0.04318
eval/accuracy,0.98597
eval/f1,0.985
eval/precision,0.98478


train/loss,█▆▃▁▁▁▁▁▁▁
train/learning_rate,▅█▇▆▅▅▄▃▂▁
train/epoch,▁▁▂▂▃▃▃▃▄▄▅▅▆▆▆▆▇▇███
train/global_step,▁▁▂▂▃▃▃▃▄▄▅▅▆▆▆▆▇▇███
_runtime,▁▁▂▂▃▃▃▃▄▄▅▅▆▆▆▆▇▇███
_timestamp,▁▁▂▂▃▃▃▃▄▄▅▅▆▆▆▆▇▇███
_step,▁▁▂▂▂▃▃▃▄▄▅▅▅▆▆▆▇▇▇██
eval/loss,█▃▁▁▁▁▁▁▁▁
eval/accuracy,▁▆████████
eval/f1,▁▇████████
eval/precision,▁▇████████


In [ ]:

#"distilbert-higher-batch-size"

training_args = TrainingArguments(
    # output directory
    output_dir=os.path.join("trained", "distilbert_cased", "results"),
    # total number of training epochs
    num_train_epochs=4,
    # batch size per device during training
    per_device_train_batch_size=32,
    # batch size for evaluation
    per_device_eval_batch_size=32,
    # number of warmup steps for learning rate scheduler
    warmup_steps=100,
    # strength of weight decay
    weight_decay=0.01, 
    # learning rate
    learning_rate=5e-5,
    # directory for storing logs
    logging_dir=os.path.join("trained", "distilbert_cased", "logs"),            
    logging_steps=50,
    evaluation_strategy = "steps",
    report_to="wandb",  # enable logging to W&B
    run_name="distilbert-higher-batch-size"
)

model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-german-cased", num_labels = len(set(test_labels)))

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

trainer.train()
wandb.finish()

Some weights of the model checkpoint at distilbert-base-german-cased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-german-cased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias

Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,N,Runtime,Samples Per Second
50,1.725700,1.458415,0.615230,0.552611,0.555986,0.596172,None,9.763600,51.108000
100,0.965300,0.470478,0.879760,0.874692,0.880135,0.877103,None,9.512100,52.459000
150,0.297000,0.120106,0.975952,0.974807,0.974264,0.975563,None,9.474400,52.668000
200,0.070400,0.066337,0.981964,0.981569,0.981192,0.982020,None,9.404400,53.060000
250,0.026700,0.071646,0.979960,0.979280,0.978870,0.979910,None,9.413600,53.009000


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


train/loss,0.0267
train/learning_rate,0.0
train/epoch,4.0
train/global_step,252
_runtime,479
_timestamp,1619974513
_step,10
eval/loss,0.07165
eval/accuracy,0.97996
eval/f1,0.97928
eval/precision,0.97887


train/loss,█▅▂▁▁
train/learning_rate,▄█▆▃▁
train/epoch,▁▁▃▃▄▄▆▆███
train/global_step,▁▁▃▃▄▄▆▆███
_runtime,▁▁▃▃▄▅▆▆███
_timestamp,▁▁▃▃▄▅▆▆███
_step,▁▂▂▃▄▅▅▆▇▇█
eval/loss,█▃▁▁▁
eval/accuracy,▁▆███
eval/f1,▁▆███
eval/precision,▁▆███


In [ ]:
#trainer.save_model(os.path.join("drive", "MyDrive", "nlpdl", "01_PPR_best_model.bin"))

## Test set performance

In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, n = precision_recall_fscore_support(labels, preds, average="none")
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall,
        'n': n
    }

In [ ]:
eval_res = trainer.predict(test_dataset)

In [ ]:
eval_res.metrics

In [ ]:
evaluated = pd.DataFrame()
evaluated["label"] = [v for k,v in test_label_dict.items()]
evaluated["class"] = [k for k,v in test_label_dict.items()]
evaluated["f1"] = eval_res.metrics["test_f1"]
evaluated["precision"] = eval_res.metrics["test_precision"]
evaluated["recall"] = eval_res.metrics["test_recall"]
evaluated["n"] = eval_res.metrics["test_n"]

evaluated.sort_values(by="f1", ascending=False)

In [ ]:
pd.crosstab(eval_res.label_ids, eval_res.predictions.argmax(-1))